<h2>This file is for processing the data from the <TT>bureau</TT> and <TT>bureau_balance</TT> data files.</h2>

In [1]:
import v10_common as com

import feather
import numpy as np
import pandas as pd
import scipy.stats as sstats

In [2]:
# Pending changes:
# - filter out not useful credit types from CREDIT_TYPE tabulation
# - filter out what's not "Active" or "Closed" from CREDIT_STATUS

In [4]:
BUREAU_FILE = com.DATA_FILE_FOLDER + "bureau.feather"
BUREAU_BALANCE_FILE = com.DATA_FILE_FOLDER + "bureau_balance.feather"

<h4>Start with the <TT>bureau_balance</TT> data, since we'll want to merge that into the <TT>bureau</TT> data pretty much immediately after we load it.</h4>

In [5]:
bureau_balance = pd.read_feather(BUREAU_BALANCE_FILE)
bureau = pd.read_feather(BUREAU_FILE)

In [6]:
bureau_balance["STATUS"] = bureau_balance["STATUS"].astype(str)
bureau_balance_gr = bureau_balance.groupby("SK_ID_BUREAU")

In [7]:
bureau_balance_sub = com.count_frac_cols(bureau_balance_gr,
                                         col = "STATUS",
                                         middle_string = "BBAL_STATUS")

In [8]:
bureau_balance_sub["BUREAU_BAL_COUNT"] = bureau_balance_gr.size()
bureau_balance_sub["MONTHS_BALANCE_MIN"] = bureau_balance_gr.agg({"MONTHS_BALANCE":min})
bureau_balance_sub["MONTHS_BALANCE_MAX"] = bureau_balance_gr.agg({"MONTHS_BALANCE":max})

In [ ]:
"""bureau_balance_n_months = []
for month in [1,6,12,24]:
    bureau_bal_recent = bureau_balance[bureau_balance["MONTHS_BALANCE"] >= -month]
    bureau_bal_recent_gr = bureau_balance_recent.groupby("SK_ID_CURR")
    
    bureau_bal_recent_sub = com.count_frac_cols(bureau_balance_gr,
                                         column_names = ["STATUS"],
                                         middle_string = f"BBAL_STATUS_{month}_MONTHS")
    bureau_bal_recent_sub[f"NUM_BBAL_ENTRIES_{month}_MONTHS"] = bureau_bal_recent_gr.size()
    
    bureau_balance_n_months.append(bureau_balance_n_months)"""

<h4>On to the <TT>bureau</TT> file.  Merge the <TT>bureau_balance</TT> data first, then we can start grouping and engineering a lot of features.</h4>

In [9]:
bureau = bureau.join(bureau_balance_sub, on = "SK_ID_BUREAU", rsuffix = "bb")

In [10]:
# usefulness of credit type is based on feature importance in lgbm; excluded ones have (near-)zero importance
useful_credit_types = ["Another type of loan", "Car loan", "Consumer credit", "Credit card",
                       "Microloan", "Mortgage"]

bureau["CREDIT_TYPE"] = bureau["CREDIT_TYPE"].astype("str")
bureau.loc[~bureau["CREDIT_TYPE"].isin(useful_credit_types),"CREDIT_TYPE"] = np.nan

In [11]:
bureau_gr = bureau.groupby("SK_ID_CURR")
bureau_sub = bureau_gr.agg({"CREDIT_DAY_OVERDUE":[max,np.mean],
                            "AMT_CREDIT_SUM_OVERDUE":[max,np.mean],
                            "CNT_CREDIT_PROLONG":sum,
                            "AMT_CREDIT_SUM":sum,
                            "AMT_CREDIT_SUM_DEBT":sum,
                            "AMT_CREDIT_SUM_LIMIT":sum,
                            "MONTHS_BALANCE_MIN":min,
                            "MONTHS_BALANCE_MAX":max,
                            "DAYS_CREDIT":[min,max,np.mean],
                            "DAYS_CREDIT_UPDATE":[min,max],
                            "DAYS_CREDIT_ENDDATE":[min,max],
                            "BUREAU_BAL_COUNT":[sum, np.mean]})
bureau_sub.columns = ["MAX_DAYS_OVERDUE", "AVG_DAYS_OVERDUE",
                      "MAX_CREDIT_OVERDUE", "AVG_CREDIT_OVERDUE",
                      "NUM_TIMES_PROLONGED",
                      "TOTAL_CURRENT_CREDIT_AMT",
                      "TOTAL_CURRENT_CREDIT_DEBT",
                      "TOTAL_CURRENT_CREDIT_LIMIT",
                      "BUREAU_OLDEST_MONTH",
                      "BUREAU_NEWEST_MONTH",
                      "BUREAU_OLDEST_APP","BUREAU_NEWEST_APP","BUREAU_AVG_APP_AGE",
                      "BUREAU_OLDEST_UPDATE","BUREAU_NEWEST_UPDATE",
                      "OLDEST_CREDIT_ENDDATE","NEWEST_CREDIT_ENDDATE",
                      "TOTAL_BUREAU_BALANCE_ENTIRES", "AVG_NUM_BUREAU_BALANCE_ENTRIES"]

bureau_sub["BUREAU_CREDIT_COUNT"] = bureau_gr.size()

In [12]:
bureau_recent_credit = bureau[["SK_ID_CURR"]].copy()
for days in [60,120,365]:
    bureau_recent_credit[f"NUM_BUREAU_APP_{days}_DAYS"] = 1 * (bureau["DAYS_CREDIT"] > -days)
bureau_recent_credit = bureau_recent_credit.groupby("SK_ID_CURR").agg({sum})
bureau_recent_credit.columns = [x[0] for x in bureau_recent_credit.columns]

In [13]:
cols = ["0","1","2","3","4","5","C","X"]
bureau_bbal_status_agg_cols = {"NUM_BBAL_STATUS_" + col:sum for col in cols}
bureau_bbal_status_agg = bureau_gr.agg(bureau_bbal_status_agg_cols)

In [14]:
bureau_credit_types = com.count_frac_cols(bureau_gr,
                                          col = "CREDIT_TYPE",
                                          middle_string = "CREDIT_TYPE")

In [15]:
bureau_credit_status = com.count_frac_cols(bureau_gr,
                                           col = "CREDIT_ACTIVE",
                                           middle_string = "CREDIT_ACTIVE")

In [16]:
bureau_data = pd.concat([bureau_sub, bureau_bbal_status_agg, bureau_credit_types, bureau_credit_status,
                         bureau_recent_credit], axis = 1)

<h4> Logistic regression predictions.  Several columns are stripped out due to multicollinearity issues (the threshold point for this is a correlation of greater than 0.75 between two variables).  A few other columns are transformed so that they have higher correlations with the TARGET variable (which seems to help the models along); since the ranking order of the points stays the same in these circumstances, the gradient boosting shouldn't be particularly affected. </h4>

<h4> LAST AUC VALUE: 0.6361 </h4>

High correlation pairs:
* MAX_DAYS_OVERDUE & AVG_DAYS_OVERDUE (0.7503)
* MAX_CREDIT_OVERDUE & AVG_CREDIT_OVERDUE (0.7803)
* BUREAU_OLDEST_APP & BUREAU_AVG_APP_AGE (0.8020)
* NUM_BUREAU_BALANCE_ENTIRES & NUM_BBAL_STATUS_C (0.9067)
* BUREAU_CREDIT_COUNT & NUM_CREDIT_ACTIVE_Closed (0.9237)
* FRAC_CREDIT_ACTIVE_Active & FRAC_CREDIT_ACTIVE_Closed (-0.9922)
* NUM_CREDIT_ACTIVE_Bad debt & FRAC_CREDIT_ACTIVE_Bad debt (0.8326)

In [17]:
#Objects needed for the logistic regression
target_df = pd.read_feather("target.feather")

bureau_poly = {"BUREAU_AVG_APP_AGE":2,"NUM_CREDIT_TYPE_Microloan":0.1}

high_cor_columns = ["AVG_DAYS_OVERDUE", "AVG_CREDIT_OVERDUE", "BUREAU_OLDEST_APP", "NUM_BBAL_STATUS_C",
                    "NUM_CREDIT_ACTIVE_Closed", "FRAC_CREDIT_ACTIVE_Closed", "FRAC_CREDIT_ACTIVE_Bad debt"]

In [18]:
# Make logistic regression predictions
test_aucs = []
for _ in range(8):
    pred, auc = com.log_regress_other_files(com.add_polynomial_terms(bureau_data.reset_index().copy(), bureau_poly),
                                            target_df,
                                            high_cor_columns)
    test_aucs.append(auc)
    print(auc)
print("Avg AUC: " + str(np.mean(test_aucs)))

0.6364224722573983
0.6355135979205379
0.6324627075689933
0.6346633885381384
0.6398315242554045
0.6318025820678381
0.635831576925115
0.6425513576178586
Avg AUC: 0.6361349008939106


In [19]:
bureau_data["BUREAU_LR_PREDS"] = pred

In [20]:
bureau_data.reset_index().to_feather("bureau_sub.feather")